In [146]:
import torch
print(torch.cuda.is_available())  
print(torch.cuda.current_device())  
print(torch.cuda.get_device_name(torch.cuda.current_device())) 

True
0
NVIDIA GeForce RTX 4060 Laptop GPU


In [147]:
import sys,os
__script_path=os.path.abspath(globals().get('__file__','.'))
__script_dir = os.path.dirname(__script_path)
root_dir = os.path.abspath(f'{__script_dir}/../../../../..')
print(root_dir)
for lib in [root_dir][::-1]:
    if lib in sys.path:
        sys.path.remove(lib)
    sys.path.insert(0,lib)
from config.config import *
# from libs.common import *

c:\Users\Admin\Data\ads_fraud_detection


In [148]:
save_dir=f"{exps_dir}/exp1/exp_smote"
if os.path.exists(save_dir) == False: 
  os.makedirs(save_dir, exist_ok=True)



test_size=0.33
seed=42

In [149]:
if torch.cuda.is_available():
    device = torch.device("cuda")
    print("CUDA is available. Using GPU.")
else:
    device = torch.device("cpu")
    print("CUDA is not available. Using CPU.")

CUDA is available. Using GPU.


In [150]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from pytorch_tabnet.pretraining import TabNetPretrainer
import torch
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, MinMaxScaler

import numpy as np


In [151]:
x_train=pd.read_excel(f'{save_dir}/x_train.xlsx')
y_train=pd.read_excel(f'{save_dir}/y_train.xlsx')
x_test=pd.read_excel(f'{save_dir}/x_test.xlsx')
y_test=pd.read_excel(f'{save_dir}/y_test.xlsx')
class_weights_dict=dict(np.load(f'{save_dir}/class_weights_dict.npz',allow_pickle=True))['class_weights_dict']
class_weights_dict = {key: value for key, value in class_weights_dict.item().items()}
class_weights_dict

{0: 0.6555772994129159, 1: 2.106918238993711}

In [152]:
X_train = torch.tensor(x_train.values, dtype=torch.float32)
y_train = torch.tensor(y_train.values, dtype=torch.long)
X_test = torch.tensor(x_test.values, dtype=torch.float32)
y_test = torch.tensor(y_test.values, dtype=torch.long)
num_classes = 2  # Assuming binary classification
y_train = F.one_hot(y_train, num_classes=num_classes)
y_test = F.one_hot(y_test, num_classes=num_classes)
print(X_train.shape, y_train.shape, X_test.shape, y_test.shape)


torch.Size([1022, 39]) torch.Size([1022, 1, 2]) torch.Size([330, 39]) torch.Size([330, 1, 2])


In [153]:
from torch.utils.data import TensorDataset, DataLoader
train_dataset = TensorDataset(X_train, y_train)
test_dataset = TensorDataset(X_test, y_test)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=True)

In [154]:
tabnet_params = {
    "n_d": 512,
    "n_a": 512,
    "n_steps": 3,
    "n_shared": 2,
    "n_independent": 2,
    "gamma": 1.3,
    "epsilon": 1e-15,
    "momentum": 0.98,
    "mask_type": "sparsemax",
    "lambda_sparse": 1e-3,
    "device_name": "cuda" if torch.cuda.is_available() else "cpu"
}


unsupervised_model = TabNetPretrainer(
    optimizer_fn=torch.optim.Adam,
    optimizer_params=dict(lr=2e-2),
    **tabnet_params
)
 

unsupervised_model.fit(
    X_train,
    eval_set=[X_test],  
    pretraining_ratio=0.8,
    max_epochs=101,
    patience=10,
    batch_size=1024,
    virtual_batch_size=128,
    num_workers=0,
    drop_last=False
)

c:\Users\Admin\miniconda3\envs\pytorch\lib\site-packages\pytorch_tabnet\abstract_model.py:82: UserWarning: Device used : cuda
  warnings.warn(f"Device used : {self.device}")


epoch 0  | loss: 29.53251| val_0_unsup_loss_numpy: 91734.125|  0:00:00s
epoch 1  | loss: 147.08255| val_0_unsup_loss_numpy: 5981.91796875|  0:00:00s
epoch 2  | loss: 65.22984| val_0_unsup_loss_numpy: 347.4918518066406|  0:00:00s
epoch 3  | loss: 21.57448| val_0_unsup_loss_numpy: 553.1861572265625|  0:00:00s
epoch 4  | loss: 15.4741 | val_0_unsup_loss_numpy: 286.1898498535156|  0:00:00s
epoch 5  | loss: 10.12379| val_0_unsup_loss_numpy: 146.8248748779297|  0:00:00s
epoch 6  | loss: 7.87988 | val_0_unsup_loss_numpy: 44.57017135620117|  0:00:00s
epoch 7  | loss: 4.95427 | val_0_unsup_loss_numpy: 44.70695877075195|  0:00:00s
epoch 8  | loss: 3.78906 | val_0_unsup_loss_numpy: 35.38682174682617|  0:00:00s
epoch 9  | loss: 3.04353 | val_0_unsup_loss_numpy: 24.55137062072754|  0:00:01s
epoch 10 | loss: 2.60534 | val_0_unsup_loss_numpy: 21.95918083190918|  0:00:01s
epoch 11 | loss: 2.40304 | val_0_unsup_loss_numpy: 20.599590301513672|  0:00:01s
epoch 12 | loss: 2.15829 | val_0_unsup_loss_numpy:

c:\Users\Admin\miniconda3\envs\pytorch\lib\site-packages\pytorch_tabnet\callbacks.py:172: UserWarning: Best weights from best epoch are automatically used!
  warnings.warn(wrn_msg)


In [155]:
# Truy cập vào mô hình TabNet bên trong
from torchinfo import summary

tabnet_model = unsupervised_model.network.to(device)

summary(tabnet_model, input_size=X_train.shape) 

Layer (type:depth-idx)                                       Output Shape              Param #
TabNetPretraining                                            [1022, 39]                --
├─EmbeddingGenerator: 1-1                                    [1022, 39]                --
├─TabNetEncoder: 1-2                                         [1022, 512]               --
│    └─BatchNorm1d: 2-1                                      [1022, 39]                78
│    └─FeatTransformer: 2-2                                  [1022, 1024]              4,202,496
│    │    └─GLU_Block: 3-1                                   [1022, 1024]              2,185,216
│    └─ModuleList: 2-12                                      --                        (recursive)
│    │    └─FeatTransformer: 3-17                            --                        (recursive)
│    └─FeatTransformer: 2-6                                  --                        (recursive)
│    │    └─GLU_Block: 3-5                            

In [156]:
encoder = tabnet_model.encoder

print("\nEncoder Summary:")
print(encoder)




Encoder Summary:
TabNetEncoder(
  (initial_bn): BatchNorm1d(39, eps=1e-05, momentum=0.01, affine=True, track_running_stats=True)
  (initial_splitter): FeatTransformer(
    (shared): GLU_Block(
      (shared_layers): ModuleList(
        (0): Linear(in_features=39, out_features=2048, bias=False)
        (1): Linear(in_features=1024, out_features=2048, bias=False)
      )
      (glu_layers): ModuleList(
        (0): GLU_Layer(
          (fc): Linear(in_features=39, out_features=2048, bias=False)
          (bn): GBN(
            (bn): BatchNorm1d(2048, eps=1e-05, momentum=0.98, affine=True, track_running_stats=True)
          )
        )
        (1): GLU_Layer(
          (fc): Linear(in_features=1024, out_features=2048, bias=False)
          (bn): GBN(
            (bn): BatchNorm1d(2048, eps=1e-05, momentum=0.98, affine=True, track_running_stats=True)
          )
        )
      )
    )
    (specifics): GLU_Block(
      (glu_layers): ModuleList(
        (0-1): 2 x GLU_Layer(
          (fc

In [157]:
decoder = tabnet_model.decoder

print("\nDecoder Summary:")
print(decoder)


Decoder Summary:
TabNetDecoder(
  (feat_transformers): ModuleList(
    (0-2): 3 x FeatTransformer(
      (shared): GLU_Block(
        (shared_layers): ModuleList(
          (0): Linear(in_features=512, out_features=1024, bias=False)
        )
        (glu_layers): ModuleList(
          (0): GLU_Layer(
            (fc): Linear(in_features=512, out_features=1024, bias=False)
            (bn): GBN(
              (bn): BatchNorm1d(1024, eps=1e-05, momentum=0.98, affine=True, track_running_stats=True)
            )
          )
        )
      )
      (specifics): GLU_Block(
        (glu_layers): ModuleList(
          (0): GLU_Layer(
            (fc): Linear(in_features=512, out_features=1024, bias=False)
            (bn): GBN(
              (bn): BatchNorm1d(1024, eps=1e-05, momentum=0.98, affine=True, track_running_stats=True)
            )
          )
        )
      )
    )
  )
  (reconstruction_layer): Linear(in_features=512, out_features=39, bias=False)
)


In [158]:
sample_input = torch.tensor(X_train[:5]).to(device)  

try:
    result = tabnet_model.encoder(sample_input)
    if isinstance(result, tuple):
        print(f'TabNet encoder trả về {len(result)} giá trị.')
        for i, res in enumerate(result):
            print(f'Giá trị {i + 1} shape: {res.shape}')
    else:
        print('TabNet encoder chỉ trả về một giá trị.')
        print(f'Giá trị shape: {result.shape}')
except Exception as e:
    print(f'Đã xảy ra lỗi: {e}')

TabNet encoder trả về 2 giá trị.
Đã xảy ra lỗi: 'list' object has no attribute 'shape'


C:\Users\Admin\AppData\Local\Temp\ipykernel_15368\3356785120.py:1: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  sample_input = torch.tensor(X_train[:5]).to(device)


In [159]:
class Sampling(nn.Module):
    def __init__(self, seed=1337):
        super(Sampling, self).__init__()
        self.seed = seed

    def forward(self, inputs):
        z_mean, z_log_var = inputs
        batch = z_mean.size(0)
        dim = z_mean.size(1)
        # print(batch, dim)
        epsilon = torch.randn(batch, dim, generator=torch.Generator().manual_seed(self.seed)).to(device)
        return z_mean + torch.exp(0.5 * z_log_var) * epsilon

In [160]:
# class VAE_Encoder(nn.Module):
#     def __init__(self, latent_dim):
#         super(VAE_Encoder, self).__init__()
#         self.tabnet_encoder = tabnet_model.encoder
#         self.mlp = nn.Sequential(
#             nn.Linear(512, 256),  
#             nn.ReLU(),
#             nn.Linear(256, 128),
#             nn.ReLU(),
#             nn.Linear(128, 96),
#             nn.ReLU(),
#             nn.Linear(96, 96),
#             nn.ReLU(),
#             nn.Linear(96, 32),
#             nn.ReLU(),
#             nn.Linear(32, latent_dim)
#         ).to(device)
#         self.fc_mean = nn.Linear(latent_dim, latent_dim).to(device)
#         self.fc_log_var = nn.Linear(latent_dim, latent_dim).to(device)
#         self.sampling = Sampling().to(device)

#     def forward(self, x):
#         x = x.to(device)
#         steps_output, _ = self.tabnet_encoder(x)
#         encoded = steps_output[-1]
#         # print("Shape of encoded tensor:", encoded.shape)
#         encoded = self.mlp(encoded)
#         z_mean = self.fc_mean(encoded)
#         z_log_var = self.fc_log_var(encoded)
#         z = self.sampling((z_mean, z_log_var))
#         # print(f'Shape of z: {z.shape} - {z_log_var.shape} -{z_log_var.shape}')
#         return z_mean, z_log_var, z


In [161]:
class VAE_Encoder(nn.Module):
    def __init__(self, latent_dim):
        super(VAE_Encoder, self).__init__()
        self.tabnet_encoder = tabnet_model.encoder

        self.mlp_0 = nn.Sequential(
            nn.Linear(512, 256),  
            nn.ReLU(),
            nn.Linear(256, 128),
            nn.ReLU(),
            nn.Linear(128, 96),
            nn.ReLU(),
            nn.Linear(96, 96),
            nn.ReLU(),
            nn.Linear(96, 32),
            nn.ReLU(),
            nn.Linear(32, latent_dim)
        ).to(device)

        self.mlp_1 = nn.Sequential(
            nn.Linear(512, 256),  
            nn.ReLU(),
            nn.Linear(256, 128),
            nn.ReLU(),
            nn.Linear(128, 96),
            nn.ReLU(),
            nn.Linear(96, 96),
            nn.ReLU(),
            nn.Linear(96, 32),
            nn.ReLU(),
            nn.Linear(32, latent_dim)
        ).to(device)

        self.fc_mean_0 = nn.Linear(latent_dim, latent_dim).to(device)
        self.fc_log_var_0 = nn.Linear(latent_dim, latent_dim).to(device)

        self.fc_mean_1 = nn.Linear(latent_dim, latent_dim).to(device)
        self.fc_log_var_1 = nn.Linear(latent_dim, latent_dim).to(device)

        self.sampling = Sampling().to(device)

    def forward(self, x, labels):
        x = x.to(device)
        labels = labels.to(device)

        steps_output, _ = self.tabnet_encoder(x)
        encoded = steps_output[-1]

        batch_size = x.size(0)
        latent_dim = self.fc_mean_0.out_features

        z_mean = torch.zeros(batch_size, latent_dim).to(device)
        z_log_var = torch.zeros(batch_size, latent_dim).to(device)
        z = torch.zeros(batch_size, latent_dim).to(device)

        idx_0 = (labels == 0)
        idx_1 = (labels == 1)

        if idx_0.sum() > 0:
            encoded_0 = encoded[idx_0]
            out_0 = self.mlp_0(encoded_0)
            mu_0 = self.fc_mean_0(out_0)
            logvar_0 = self.fc_log_var_0(out_0)
            z_0 = self.sampling((mu_0, logvar_0))

            z_mean[idx_0] = mu_0
            z_log_var[idx_0] = logvar_0
            z[idx_0] = z_0

        if idx_1.sum() > 0:
            encoded_1 = encoded[idx_1]
            out_1 = self.mlp_1(encoded_1)
            mu_1 = self.fc_mean_1(out_1)
            logvar_1 = self.fc_log_var_1(out_1)
            z_1 = self.sampling((mu_1, logvar_1))

            z_mean[idx_1] = mu_1
            z_log_var[idx_1] = logvar_1
            z[idx_1] = z_1

        return z_mean, z_log_var, z


In [162]:
class VAE_Decoder(nn.Module):
    def __init__(self, latent_dim,encoded_dim, output_dim):
        super(VAE_Decoder, self).__init__()
        self.mlp = nn.Sequential(
            nn.Linear(latent_dim, 32),   
            nn.ReLU(),
            nn.Linear(32, 96),
            nn.ReLU(),
            nn.Linear(96, 96),
            nn.ReLU(),
            nn.Linear(96, 128),
            nn.ReLU(),
            nn.Linear(128, 256),
            nn.ReLU(),
            nn.Linear(256, encoded_dim),  
        )
        self.tabnet_decoder = tabnet_model.decoder
        self.reshape = nn.Unflatten(1, (encoded_dim,))
        self.output_dim=output_dim


    def forward(self, z):
        x = F.relu(self.mlp(z))

        # print("Shape before reshape:", x.shape)
        x = self.reshape(x)
        x = x[None, ...]

        # print("Shape after reshape:", x.shape)
        # x = x.view(x.size(0), output_dim)
        
        output = self.tabnet_decoder(x)
        # print(output.shape)
        # print("Shape of output from tabnet_decoder:", output.shape)
        output = torch.softmax(output, dim=-1)  # Assuming the output is a probability distribution
        output = output.view(-1, self.output_dim)
        return output

In [163]:
def check_data_range(tensor, name):
    if not torch.all((tensor >= 0) & (tensor <= 1)):
        print(f"{name} contains values outside the range [0, 1]")
        print(f"{name} min: {tensor.min()}, max: {tensor.max()}")

In [164]:
class VAE_Tabnet_MLPS(nn.Module):
    def __init__(self, encoder, decoder, classifier):
        super(VAE_Tabnet_MLPS, self).__init__()
        self.encoder = encoder
        self.decoder = decoder
        self.classifier = classifier
        self.total_loss_tracker = []
        self.reconstruction_loss_tracker = []
        self.kl_loss_tracker = []
        self.classification_loss_tracker = []
        self.accuracy_tracker = []

    def forward(self, x, labels):
        z_mean, z_log_var, z = self.encoder(x, labels)
        reconstruction = self.decoder(z)
        classification_output = self.classifier(z)
        return reconstruction, z_mean, z_log_var, classification_output

    def train_step(self, data, labels, optimizer):
        labels = labels.squeeze(1) 
        labels = torch.argmax(labels, dim=1).long()
        labels = labels.to(device)
        # print(f"Shape of labels: {labels.shape}")
        # optimizer.zero_grad()
        # z_mean, z_log_var, z = self.encoder(data)
        # reconstruction = self.decoder(z)
        reconstruction, z_mean, z_log_var, classification_output = self.forward(data, labels)
        # print('classifi',classification_output.shape)
        # print(check_data_range(data, 'data'))
        # print(check_data_range(reconstruction, 'reconstruction'))
        # reconstruction_loss = torch.mean(
        #     torch.sum(
        #         F.binary_cross_entropy(reconstruction, data, reduction='none'),
        #         dim=1
        #     )
        # )
        reconstruction_loss = torch.mean(
            torch.sum(
                F.binary_cross_entropy_with_logits(reconstruction, data, reduction='none'),
                dim=1
                # dim=(1, 2)
                )  
        )

        # print("labels:", labels)
        # print("labels.shape:", labels.shape)
        # print("labels.dtype:", labels.dtype)
        # print("labels.min():", labels.min().item(), "labels.max():", labels.max().item())
        # print("classification_output.shape:", classification_output.shape)

        classification_loss = F.cross_entropy(classification_output, labels)
                # dim=1
                # dim=(1, 2)
                
        
        kl_loss = -0.5 * torch.sum(1 + z_log_var - z_mean.pow(2) - z_log_var.exp(), dim=1)
        kl_loss = torch.mean(torch.sum(kl_loss))
        total_loss = reconstruction_loss + kl_loss + classification_loss
        total_loss.backward()
        optimizer.step()

        self.total_loss_tracker.append(total_loss.item())
        self.reconstruction_loss_tracker.append(reconstruction_loss.item())
        self.kl_loss_tracker.append(kl_loss.item())
        self.classification_loss_tracker.append(classification_loss.item())
        # print(classification_output.shape, labels.shape)

        preds = torch.softmax(classification_output, dim=1)
        pred_labels = torch.argmax(preds, dim=1)
        correct = (pred_labels == labels).float().sum()
        accuracy = correct / labels.size(0)
        self.accuracy_tracker.append(accuracy.item())

        return {
            "loss": total_loss.item(),
            "reconstruction_loss": reconstruction_loss.item(),
            "kl_loss": kl_loss.item(),
            "classification_loss": classification_loss.item(),
            "accuracy": accuracy.item()
        }

In [165]:
latent_dim = 64
encoded_dim = 512
output_dim = X_train.shape[1]
input_dim = X_train.shape[1]
print(input_dim)


39


In [166]:
class SimpleClassifier(nn.Module):
    def __init__(self, input_dim, output_dim):
        super(SimpleClassifier, self).__init__()
        self.fc_layers = nn.Sequential(
            nn.Linear(input_dim, 256),
            nn.BatchNorm1d(256),
            nn.ReLU(),
            nn.Dropout(0.3),

            nn.Linear(256, 128),
            nn.BatchNorm1d(128),
            nn.ReLU(),
            nn.Dropout(0.3),

            nn.Linear(128, 64),
            nn.BatchNorm1d(64),
            nn.ReLU(),
            nn.Dropout(0.2),

            nn.Linear(64, 32),
            nn.BatchNorm1d(32),
            nn.ReLU(),
            nn.Dropout(0.1),

            nn.Linear(32, output_dim),
            # nn.Softmax()
        )

    def forward(self, x):
        # print('input: ',x.shape)
        output = self.fc_layers(x)
        # output = output.view(-1)
        # print('output',output.shape)
        return output

In [167]:
classifier = SimpleClassifier(latent_dim, output_dim=2).to(device)


In [168]:
def check_output(model, input_tensor):
    with torch.no_grad():  
        output = model(input_tensor)
        print(f"Input size: {input_tensor.size()}")
        print(f"Output size: {output.size()}")
        print(f"Output: {output}")

model = SimpleClassifier(latent_dim, output_dim=1)

input_tensor = torch.randn(32,latent_dim)  

check_output(model, input_tensor)

Input size: torch.Size([32, 64])
Output size: torch.Size([32, 1])
Output: tensor([[ 0.2251],
        [-0.0212],
        [-1.0684],
        [-0.6204],
        [-0.8287],
        [ 0.1150],
        [-0.6822],
        [-0.4576],
        [-0.5331],
        [-0.1956],
        [-0.1393],
        [-0.2319],
        [-0.4192],
        [-0.3321],
        [ 0.0387],
        [ 0.1634],
        [-1.0403],
        [-0.5110],
        [-0.7737],
        [ 0.4728],
        [-0.9170],
        [-1.0290],
        [-0.2032],
        [-0.1186],
        [-0.6675],
        [-0.1837],
        [-0.2925],
        [-0.0863],
        [-0.5102],
        [-0.2662],
        [-0.2039],
        [-0.2969]])


In [169]:
vae_encoder = VAE_Encoder(latent_dim=latent_dim)

In [170]:
# vae_encoder = VAE_Encoder(latent_dim=latent_dim)
# print("Encoder Summary:")
# # vae_encoder.to(device)

# summary(vae_encoder, input_size=(32, input_dim), device=device)

In [171]:
x = torch.randn(800,39 ).to(device)
steps_output, _ = tabnet_model.encoder(x)
encoded = steps_output[-1]
print(f"Encoded shape: {encoded.shape}")

Encoded shape: torch.Size([800, 512])


In [172]:
import torch

x = torch.randn(800, 39).to(device)  # Đầu vào có kích thước (batch_size, features)

steps_output, _ = tabnet_model.encoder(x)
print("Shape of encoder output:", [output.shape for output in steps_output])

decoder_input = steps_output[-1]  
decoder_input = decoder_input[None, ...]
try:
    decoder_output = tabnet_model.decoder(decoder_input)
    print(f"Decoder shape: {decoder_output.shape}")
except ValueError as e:
    print(f"Error: {e}")


Shape of encoder output: [torch.Size([800, 512]), torch.Size([800, 512]), torch.Size([800, 512])]
Decoder shape: torch.Size([800, 39])


In [173]:
encoded_dim, output_dim

(512, 39)

In [174]:
vae_decoder = VAE_Decoder(latent_dim=latent_dim, encoded_dim=encoded_dim, output_dim=output_dim).to(device)
print("Decoder Summary:")
summary(vae_decoder, input_size=(32, latent_dim), device=device)

Decoder Summary:


Layer (type:depth-idx)                                       Output Shape              Param #
VAE_Decoder                                                  [32, 39]                  --
├─Sequential: 1-1                                            [32, 512]                 --
│    └─Linear: 2-1                                           [32, 32]                  2,080
│    └─ReLU: 2-2                                             [32, 32]                  --
│    └─Linear: 2-3                                           [32, 96]                  3,168
│    └─ReLU: 2-4                                             [32, 96]                  --
│    └─Linear: 2-5                                           [32, 96]                  9,312
│    └─ReLU: 2-6                                             [32, 96]                  --
│    └─Linear: 2-7                                           [32, 128]                 12,416
│    └─ReLU: 2-8                                             [32, 128]            

In [175]:
vae = VAE_Tabnet_MLPS(encoder=vae_encoder, decoder=vae_decoder,classifier=classifier).to(device)
# summary(vae, input_size=(32, input_dim), device=device)

In [176]:
learning_rate = 0.0001
optimizer = optim.Adam(vae.parameters(), lr=learning_rate)
num_epochs = 10

for epoch in range(num_epochs):
    vae.train()
    train_loss = 0
    rec_loss = 0
    kl_loss = 0
    classification_loss = 0
    accuracy = 0

    for batch_data, batch_labels in train_loader:
        batch_data = batch_data.to(device)
        batch_labels = batch_labels.to(device)
        # print(f"Batch data shape: {batch_data.shape}, Batch labels shape: {batch_labels.shape}")
        results = vae.train_step(batch_data, batch_labels, optimizer)
        
        train_loss += results["loss"]
        rec_loss += results["reconstruction_loss"]
        kl_loss += results["kl_loss"]
        classification_loss += results["classification_loss"]
        accuracy += results["accuracy"]

    train_loss /= len(train_loader)
    rec_loss /= len(train_loader)
    kl_loss /= len(train_loader)
    classification_loss /= len(train_loader)
    accuracy /= len(train_loader)

    print(f"Epoch {epoch + 1}/{num_epochs}, Loss: {train_loss:.4f}, Reconstruction Loss: {rec_loss:.4f}, KL Loss: {kl_loss:.4f}, Classification Loss: {classification_loss:.4f}, Accuracy: {accuracy:.4f}")


Epoch 1/10, Loss: 22.5496, Reconstruction Loss: 11.7630, KL Loss: 10.0801, Classification Loss: 0.7065, Accuracy: 0.5167
Epoch 2/10, Loss: 15.7452, Reconstruction Loss: 10.0034, KL Loss: 5.0573, Classification Loss: 0.6845, Accuracy: 0.5498
Epoch 3/10, Loss: 12.0356, Reconstruction Loss: 9.1090, KL Loss: 2.2242, Classification Loss: 0.7024, Accuracy: 0.5421
Epoch 4/10, Loss: 10.8429, Reconstruction Loss: 8.7973, KL Loss: 1.3292, Classification Loss: 0.7165, Accuracy: 0.5068
Epoch 5/10, Loss: 10.6935, Reconstruction Loss: 8.7474, KL Loss: 1.2341, Classification Loss: 0.7119, Accuracy: 0.5040
Epoch 6/10, Loss: 10.7532, Reconstruction Loss: 8.7077, KL Loss: 1.3402, Classification Loss: 0.7054, Accuracy: 0.5027
Epoch 7/10, Loss: 10.7184, Reconstruction Loss: 8.7038, KL Loss: 1.3126, Classification Loss: 0.7020, Accuracy: 0.5087
Epoch 8/10, Loss: 10.6552, Reconstruction Loss: 8.6939, KL Loss: 1.2567, Classification Loss: 0.7046, Accuracy: 0.4934
Epoch 9/10, Loss: 10.5301, Reconstruction Los

In [177]:
for param in vae.encoder.parameters():
    param.requires_grad = False

In [178]:
vae_new = VAE_Tabnet_MLPS(vae.encoder, vae.decoder, vae.classifier).to(device)
for param in vae_new.encoder.parameters():
    param.requires_grad = False

optimizer = optim.Adam(filter(lambda p: p.requires_grad, vae_new.parameters()), lr=learning_rate)
for epoch in range(num_epochs):
    vae_new.train()
    train_loss = 0
    rec_loss = 0
    kl_loss = 0
    classification_loss = 0
    accuracy = 0

    for batch_data, batch_labels in train_loader:
        batch_data = batch_data.to(device)
        batch_labels = batch_labels.to(device)
        results = vae.train_step(batch_data, batch_labels, optimizer)
        
        train_loss += results["loss"]
        rec_loss += results["reconstruction_loss"]
        kl_loss += results["kl_loss"]
        classification_loss += results["classification_loss"]
        accuracy += results["accuracy"]

    train_loss /= len(train_loader)
    rec_loss /= len(train_loader)
    kl_loss /= len(train_loader)
    classification_loss /= len(train_loader)
    accuracy /= len(train_loader)

    print(f"Epoch {epoch + 1}/{num_epochs}, Loss: {train_loss:.4f}, Reconstruction Loss: {rec_loss:.4f}, KL Loss: {kl_loss:.4f}, Classification Loss: {classification_loss:.4f}, Accuracy: {accuracy:.4f}")

Epoch 1/10, Loss: 10.3270, Reconstruction Loss: 8.6919, KL Loss: 0.9367, Classification Loss: 0.6984, Accuracy: 0.4913
Epoch 2/10, Loss: 10.3347, Reconstruction Loss: 8.7012, KL Loss: 0.9367, Classification Loss: 0.6968, Accuracy: 0.5119
Epoch 3/10, Loss: 10.3273, Reconstruction Loss: 8.6925, KL Loss: 0.9362, Classification Loss: 0.6986, Accuracy: 0.4904
Epoch 4/10, Loss: 10.3295, Reconstruction Loss: 8.6983, KL Loss: 0.9363, Classification Loss: 0.6950, Accuracy: 0.4921
Epoch 5/10, Loss: 10.3268, Reconstruction Loss: 8.6959, KL Loss: 0.9373, Classification Loss: 0.6936, Accuracy: 0.5060
Epoch 6/10, Loss: 10.3248, Reconstruction Loss: 8.6923, KL Loss: 0.9364, Classification Loss: 0.6960, Accuracy: 0.4930
Epoch 7/10, Loss: 10.3202, Reconstruction Loss: 8.6888, KL Loss: 0.9362, Classification Loss: 0.6952, Accuracy: 0.4862
Epoch 8/10, Loss: 10.3277, Reconstruction Loss: 8.6999, KL Loss: 0.9370, Classification Loss: 0.6908, Accuracy: 0.5302
Epoch 9/10, Loss: 10.3282, Reconstruction Loss: 

In [179]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class SimpleDiffusionModel(nn.Module):
    def __init__(self, latent_dim, time_steps=1000):
        super().__init__()
        self.time_steps = time_steps
        self.latent_dim = latent_dim
        
        # Tạo các beta_schedule tuyến tính
        beta = torch.linspace(0.0001, 0.02, time_steps)
        alpha = 1. - beta
        alpha_bar = torch.cumprod(alpha, dim=0)

        self.register_buffer('beta', beta)
        self.register_buffer('alpha', alpha)
        self.register_buffer('alpha_bar', alpha_bar)

        # Mạng neural đơn giản để dự đoán nhiễu
        self.model = nn.Sequential(
            nn.Linear(latent_dim + 1, 512),
            nn.ReLU(),
            nn.Linear(512, latent_dim)
        )

    def forward(self, z, t):
        noise = torch.randn_like(z)
        
        # Đảm bảo t là long type và shape phù hợp
        if isinstance(t, torch.Tensor):
            t = t.to(dtype=torch.long)
        else:
            t = torch.tensor([t], device=z.device, dtype=torch.long).expand(z.shape[0])

        sqrt_alpha_bar = torch.sqrt(self.alpha_bar[t])[:, None]
        sqrt_one_minus_alpha_bar = torch.sqrt(1 - self.alpha_bar[t])[:, None]
        noisy_z = sqrt_alpha_bar * z + sqrt_one_minus_alpha_bar * noise

        predicted_noise = self.model(torch.cat([noisy_z, t.unsqueeze(1)], dim=1))
        loss = F.mse_loss(predicted_noise, noise)
        return loss

    def sample(self, num_samples):
        z = torch.randn(num_samples, self.latent_dim).to(next(self.parameters()).device)
        for i in reversed(range(self.time_steps)):
            t = torch.full((num_samples,), i, device=z.device, dtype=torch.long)
            z = self.denoise_step(z, t)
        return z
    
    def denoise_step(self, z, t):
        timestep = t.item() if isinstance(t, torch.Tensor) else t
        t_batch = torch.full((z.shape[0],), timestep, device=z.device, dtype=torch.long)

        predicted_noise = self.model(torch.cat([z, t_batch.unsqueeze(1)], dim=1))

        alpha = self.alpha[timestep]
        alpha_bar = self.alpha_bar[timestep]
        beta = self.beta[timestep]

        z = (1 / torch.sqrt(alpha)) * (z - ((1 - alpha) / torch.sqrt(1 - alpha_bar)) * predicted_noise)
        if timestep > 0:
            noise = torch.randn_like(z)
            z += torch.sqrt(beta) * noise
        return z

In [180]:
# Thay đoạn lỗi này:
# latent_dim = vae.encoder[-1].out_features

# Bằng đoạn này:
with torch.no_grad():
    vae_new.encoder.eval()
    dummy_input = torch.randn(1, input_dim).to(device)  # Thay input_dim theo đúng dữ liệu của bạn
    dummy_labels = torch.zeros(1, dtype=torch.long).to(device)
    z_mean, z_log_var, _ = vae_new.encoder(dummy_input, dummy_labels)
    latent_dim = z_mean.shape[1]

In [181]:
# latent_dim = vae.encoder[-1].out_features  # Kích thước latent z
diffusion_model = SimpleDiffusionModel(latent_dim=latent_dim).to(device)
diffusion_optimizer = torch.optim.Adam(diffusion_model.parameters(), lr=1e-3)

In [182]:
from tqdm import tqdm
def train_diffusion(vae, diffusion_model, dataloader, optimizer, device, time_steps=1000, epochs=150):
    diffusion_model.train()
    for epoch in range(epochs):
        total_loss = 0
        for batch_data, batch_labels in tqdm(dataloader, desc=f"Epoch {epoch+1}/{epochs}"):
            batch_data = batch_data.to(device)
            batch_labels = batch_labels.squeeze().to(device)
            if batch_labels.dim() > 1:  # nếu là one-hot hoặc có chiều phụ
                batch_labels = batch_labels.argmax(dim=1)
            with torch.no_grad():
                z_mean, z_log_var, z = vae.encoder(batch_data, batch_labels)

            t = torch.randint(0, time_steps, (z.shape[0],), device=device).long()
            loss = diffusion_model(z, t)

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            total_loss += loss.item()

        print(f"Epoch {epoch+1}, Diffusion Loss: {total_loss/len(dataloader):.4f}")

In [183]:
def evaluate_diffusion_with_classifier(vae, diffusion_model, classifier, test_loader, device, time_steps=1000):
    vae.eval()
    diffusion_model.eval()
    classifier.eval()
    
    correct = 0
    total = 0
    
    with torch.no_grad():
        for batch_data, batch_labels in test_loader:
            batch_data = batch_data.to(device)
            batch_labels = batch_labels.squeeze().to(device)
            if batch_labels.ndim == 3 and batch_labels.shape[1] == 1:
                batch_labels = batch_labels.squeeze(1)  # bỏ chiều 1

            if batch_labels.ndim == 2 and batch_labels.shape[1] == 2:
                batch_labels = torch.argmax(batch_labels, dim=1)
            elif batch_labels.ndim == 2 and batch_labels.shape[1] == 1:
                batch_labels = batch_labels.squeeze(1)

            z_mean, z_log_var, z = vae.encoder(batch_data, batch_labels)

            t_forward = time_steps - 1
            sqrt_alpha_bar = torch.sqrt(diffusion_model.alpha_bar[t_forward])
            sqrt_one_minus_alpha_bar = torch.sqrt(1 - diffusion_model.alpha_bar[t_forward])
            noisy_z = sqrt_alpha_bar * z + sqrt_one_minus_alpha_bar * torch.randn_like(z)

            z_recovered = noisy_z
            for t in reversed(range(time_steps)):
                z_recovered = diffusion_model.denoise_step(z_recovered, t)
            logits = classifier(z_recovered)

            # print("Logits shape:", logits.shape)  # Debug

            if len(logits.shape) == 1:
                preds = (torch.sigmoid(logits) > 0.5).float()
            elif len(logits.shape) == 2:
                preds = torch.argmax(logits, dim=1)
            else:
                raise ValueError(f"Unexpected logits shape: {logits.shape}")

            # print(f"preds shape: {preds.shape}, batch_labels shape: {batch_labels.shape}")  # Debug

            correct += (preds == batch_labels).sum().item()
            total += batch_labels.size(0)

    accuracy = correct / total
    print(f"Accuracy on recovered z: {accuracy:.4f}")


In [184]:
device

device(type='cuda')

In [185]:
# latent_dim = vae.encoder[-1].out_features
diffusion_model = SimpleDiffusionModel(latent_dim=latent_dim).to(device)
diffusion_optimizer = optim.Adam(diffusion_model.parameters(), lr=1e-3)

train_diffusion(vae_new, diffusion_model, train_loader, diffusion_optimizer, device)

evaluate_diffusion_with_classifier(vae, diffusion_model, vae.classifier, test_loader, device)

Epoch 1/150: 100%|██████████| 32/32 [00:00<00:00, 103.74it/s]


Epoch 1, Diffusion Loss: 33.0828


Epoch 2/150: 100%|██████████| 32/32 [00:00<00:00, 102.62it/s]


Epoch 2, Diffusion Loss: 1.9369


Epoch 3/150: 100%|██████████| 32/32 [00:00<00:00, 109.91it/s]


...

Epoch 148, Diffusion Loss: 0.1523


Epoch 149/150: 100%|██████████| 32/32 [00:00<00:00, 107.95it/s]


Epoch 149, Diffusion Loss: 0.1593


Epoch 150/150: 100%|██████████| 32/32 [00:00<00:00, 104.50it/s]


Epoch 150, Diffusion Loss: 0.1449
Accuracy on recovered z: 0.5394
